# IMPORT LIBRARIES

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import seaborn as sns
import os


import warnings
warnings.filterwarnings('ignore')
pd.options.display.max_seq_items = 500
pd.options.display.max_rows = 500

# LOAD DATA

In [ ]:
app_train = pd.read_csv('app_train_cleaned.csv')
app_test = pd.read_csv('app_test_cleaned.csv')
bureau = pd.read_csv('bureau.csv')
bureau_blnc = pd.read_csv('bureau_balance.csv')
prvs_app = pd.read_csv('previous_application.csv')
pos_cash = pd.read_csv('POS_CASH_balance.csv')
ins_pay = pd.read_csv('installments_payments.csv')
cc_blnc = pd.read_csv('credit_card_balance.csv')

In [ ]:
app_train.shape

(304527, 70)

## 3.2 Construct Required Data

### 3.2.1 Data app_train

In [ ]:
#Flag untuk mewakili saat Total pendapatan lebih besar dari Kredit
app_train['INCOME_>_CREDIT_FLAG'] = app_train['AMT_INCOME_TOTAL'] > app_train['AMT_CREDIT']

In [ ]:
#Kolom untuk mewakili Persentase Pendapatan Kredit
app_train['CREDIT_INCOME_%'] = app_train['AMT_CREDIT'] / app_train['AMT_INCOME_TOTAL']

In [ ]:
#Kolom untuk mewakili Persentase Pendapatan Anuitas
app_train['ANNUITY_INCOME_%'] = app_train['AMT_ANNUITY'] / app_train['AMT_INCOME_TOTAL']

In [ ]:
#Kolom untuk mewakili Jangka Kredit
app_train['CREDIT_TERM'] = app_train['AMT_CREDIT'] / app_train['AMT_ANNUITY'] 

In [ ]:
#Kolom untuk mewakili Persentase Hari Bekerja dalam hidupnya
app_train['DAYS_EMPLOYED_%'] = app_train['DAYS_EMPLOYED'] / app_train['DAYS_BIRTH']

In [ ]:
#Bentuk (shape) dari data Aplikasi
print('The shape of app_train data:',app_train.shape)

The shape of app_train data: (304527, 75)


### 3.2.2 Data app__test

In [ ]:
#Flag untuk mewakili saat Total pendapatan lebih besar dari Kredit
app_test['INCOME_>_CREDIT_FLAG'] = app_test['AMT_INCOME_TOTAL'] > app_test['AMT_CREDIT']

In [ ]:
#Kolom untuk mewakili Persentase Pendapatan Kredit
app_test['CREDIT_INCOME_%'] = app_test['AMT_CREDIT'] / app_test['AMT_INCOME_TOTAL']

In [ ]:
#Kolom untuk mewakili Persentase Pendapatan Anuitas
app_test['ANNUITY_INCOME_%'] = app_test['AMT_ANNUITY'] / app_test['AMT_INCOME_TOTAL']

In [ ]:
#Kolom untuk mewakili Jangka Kredit
app_test['CREDIT_TERM'] = app_test['AMT_CREDIT'] / app_test['AMT_ANNUITY'] 

In [ ]:
#Kolom untuk mewakili Persentase Hari Bekerja dalam hidupnya
app_test['DAYS_EMPLOYED_%'] = app_test['DAYS_EMPLOYED'] / app_test['DAYS_BIRTH']

In [ ]:
#Bentuk (shape) dari data Aplikasi
print('The shape of app_train data:',app_test.shape)

The shape of app_train data: (47772, 74)


## 3.3 Integrate Data  


### 3.3.1 Data app_train

#### 3.3.1.1 Joining bureau w/ app_train

In [ ]:
print('Shape dari Bureau:', bureau.shape)
bureau.head()

Shape dari Bureau: (1450019, 17)


,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
0,215354,5714462,Closed,currency 1,-497.0,0.0,-153.0,-153.0,NaN,0.0,91323.0,0.0,NaN,0.0,Consumer credit,-131.0,NaN
1,215354,5714463,Active,currency 1,-208.0,0.0,1075.0,NaN,NaN,0.0,225000.0,171342.0,NaN,0.0,Credit card,-20.0,NaN
2,215354,5714464,Active,currency 1,-203.0,0.0,528.0,NaN,NaN,0.0,464323.5,NaN,NaN,0.0,Consumer credit,-16.0,NaN
3,215354,5714465,Active,currency 1,-203.0,0.0,NaN,NaN,NaN,0.0,90000.0,NaN,NaN,0.0,Credit card,-16.0,NaN
4,215354,5714466,Active,currency 1,-629.0,0.0,1197.0,NaN,77674.5,0.0,2700000.0,NaN,NaN,0.0,Consumer credit,-21.0,NaN


In [ ]:
# Combining numerical features
grp = bureau.drop(['SK_ID_BUREAU'], axis = 1).groupby(by=['SK_ID_CURR']).mean().reset_index()
grp.columns = ['BUREAU_'+ column if column !='SK_ID_CURR' else column for column in grp.columns]
apptrain_bureau = app_train.merge(grp, on='SK_ID_CURR', how='left')
apptrain_bureau.update(apptrain_bureau[grp.columns].fillna(0))

In [ ]:
#combining categorical features
cats_br = pd.get_dummies(bureau.select_dtypes('object'))
cats_br['SK_ID_CURR'] = bureau['SK_ID_CURR']

grp = cats_br.groupby(by = ['SK_ID_CURR']).mean().reset_index()
grp.columns = ['BUREAU_'+column if column !='SK_ID_CURR' else column for column in grp.columns]
apptrain_bureau = apptrain_bureau.merge(grp, on='SK_ID_CURR', how='left')
apptrain_bureau.update(apptrain_bureau[grp.columns].fillna(0))

In [ ]:
# Shape of application and bureau data combined
print('The shape application and bureau data combined:',apptrain_bureau.shape)

The shape application and bureau data combined: (304527, 110)


**Feature Engineering of apptrain_bureau Data:**

In [ ]:
# Number of past loans per customer
grp = bureau.groupby(by = ['SK_ID_CURR'])['SK_ID_BUREAU'].count().reset_index().rename(columns = {'SK_ID_BUREAU': 'BUREAU_LOAN_COUNT'})

apptrain_bureau = apptrain_bureau.merge(grp, on='SK_ID_CURR', how='left')
apptrain_bureau['BUREAU_LOAN_COUNT'] = apptrain_bureau['BUREAU_LOAN_COUNT'].fillna(0)

In [ ]:
# Number of types of past loans per customer 
grp = bureau[['SK_ID_CURR', 'CREDIT_TYPE']].groupby(by = ['SK_ID_CURR'])['CREDIT_TYPE'].nunique().reset_index().rename(columns={'CREDIT_TYPE': 'BUREAU_LOAN_TYPES'})

apptrain_bureau = apptrain_bureau.merge(grp, on='SK_ID_CURR', how='left')
apptrain_bureau['BUREAU_LOAN_TYPES'] = apptrain_bureau['BUREAU_LOAN_TYPES'].fillna(0)

In [ ]:
# Debt over credit ratio 
bureau['AMT_CREDIT_SUM'] = bureau['AMT_CREDIT_SUM'].fillna(0)
bureau['AMT_CREDIT_SUM_DEBT'] = bureau['AMT_CREDIT_SUM_DEBT'].fillna(0)

grp1 = bureau[['SK_ID_CURR','AMT_CREDIT_SUM']].groupby(by=['SK_ID_CURR'])['AMT_CREDIT_SUM'].sum().reset_index().rename(columns={'AMT_CREDIT_SUM': 'TOTAL_CREDIT_SUM'})
grp2 = bureau[['SK_ID_CURR','AMT_CREDIT_SUM_DEBT']].groupby(by=['SK_ID_CURR'])['AMT_CREDIT_SUM_DEBT'].sum().reset_index().rename(columns={'AMT_CREDIT_SUM_DEBT':'TOTAL_CREDIT_SUM_DEBT'})

grp1['DEBT_CREDIT_RATIO'] = grp2['TOTAL_CREDIT_SUM_DEBT']/grp1['TOTAL_CREDIT_SUM']
del grp1['TOTAL_CREDIT_SUM']

apptrain_bureau = apptrain_bureau.merge(grp1, on='SK_ID_CURR', how='left')
apptrain_bureau['DEBT_CREDIT_RATIO'] = apptrain_bureau['DEBT_CREDIT_RATIO'].fillna(0)
apptrain_bureau['DEBT_CREDIT_RATIO'] = apptrain_bureau['DEBT_CREDIT_RATIO'].replace([np.inf, -np.inf], 0)
apptrain_bureau['DEBT_CREDIT_RATIO'] = pd.to_numeric(apptrain_bureau['DEBT_CREDIT_RATIO'], downcast='float')

In [ ]:
# Overdue over debt ratio
bureau['AMT_CREDIT_SUM_OVERDUE'] = bureau['AMT_CREDIT_SUM_OVERDUE'].fillna(0)
bureau['AMT_CREDIT_SUM_DEBT'] = bureau['AMT_CREDIT_SUM_DEBT'].fillna(0)

grp1 = bureau[['SK_ID_CURR','AMT_CREDIT_SUM_OVERDUE']].groupby(by=['SK_ID_CURR'])['AMT_CREDIT_SUM_OVERDUE'].sum().reset_index().rename(columns={'AMT_CREDIT_SUM_OVERDUE': 'TOTAL_CUSTOMER_OVERDUE'})
grp2 = bureau[['SK_ID_CURR','AMT_CREDIT_SUM_DEBT']].groupby(by=['SK_ID_CURR'])['AMT_CREDIT_SUM_DEBT'].sum().reset_index().rename(columns={'AMT_CREDIT_SUM_DEBT':'TOTAL_CUSTOMER_DEBT'})

grp1['OVERDUE_DEBT_RATIO'] = grp1['TOTAL_CUSTOMER_OVERDUE']/grp2['TOTAL_CUSTOMER_DEBT']
del grp1['TOTAL_CUSTOMER_OVERDUE']

apptrain_bureau = apptrain_bureau.merge(grp1, on='SK_ID_CURR', how='left')
apptrain_bureau['OVERDUE_DEBT_RATIO'] =apptrain_bureau['OVERDUE_DEBT_RATIO'].fillna(0)
apptrain_bureau['OVERDUE_DEBT_RATIO'] = apptrain_bureau['OVERDUE_DEBT_RATIO'].replace([np.inf, -np.inf], 0)
apptrain_bureau['OVERDUE_DEBT_RATIO'] = pd.to_numeric(apptrain_bureau['OVERDUE_DEBT_RATIO'], downcast='float')

#### 3.3.1.2 Joining prvs_app w/ apptrain_bureau

In [ ]:
print('Shape dari prvs_app:', prvs_app.shape)
prvs_app.head()

Shape dari prvs_app: (595641, 37)


,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,...,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
0,2030495,271877,Consumer loans,1730.430,17145.0,17145.0,0.0,17145.0,SATURDAY,15.0,...,Connectivity,12.0,middle,POS mobile with interest,365243.0,-42.0,300.0,-42.0,-37.0,0.0
1,2802425,108129,Cash loans,25188.615,607500.0,679671.0,NaN,607500.0,THURSDAY,11.0,...,XNA,36.0,low_action,Cash X-Sell: low,365243.0,-134.0,916.0,365243.0,365243.0,1.0
2,2523466,122040,Cash loans,15060.735,112500.0,136444.5,NaN,112500.0,TUESDAY,11.0,...,XNA,12.0,high,Cash X-Sell: high,365243.0,-271.0,59.0,365243.0,365243.0,1.0
3,2819243,176158,Cash loans,47041.335,450000.0,470790.0,NaN,450000.0,MONDAY,7.0,...,XNA,12.0,middle,Cash X-Sell: middle,365243.0,-482.0,-152.0,-182.0,-177.0,1.0
4,1784265,202054,Cash loans,31924.395,337500.0,404055.0,NaN,337500.0,THURSDAY,9.0,...,XNA,24.0,high,Cash Street: high,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Number of previous applications per customer
grp = prvs_app[['SK_ID_CURR','SK_ID_PREV']].groupby(by=['SK_ID_CURR'])['SK_ID_PREV'].count().reset_index().rename(columns={'SK_ID_PREV':'PREV_APP_COUNT'})
apptrain_bureau_prev = apptrain_bureau.merge(grp, on =['SK_ID_CURR'], how = 'left')
apptrain_bureau_prev['PREV_APP_COUNT'] = apptrain_bureau_prev['PREV_APP_COUNT'].fillna(0)

In [ ]:
# Combining numerical features
grp = prvs_app.drop('SK_ID_PREV', axis =1).groupby(by=['SK_ID_CURR']).mean().reset_index()
prev_columns = ['PREV_'+column if column != 'SK_ID_CURR' else column for column in grp.columns ]
grp.columns = prev_columns
apptrain_bureau_prev = apptrain_bureau_prev.merge(grp, on =['SK_ID_CURR'], how = 'left')
apptrain_bureau_prev.update(apptrain_bureau_prev[grp.columns].fillna(0))

In [ ]:
# Combining categorical features
cats_prev = pd.get_dummies(prvs_app.select_dtypes('object'))
cats_prev['SK_ID_CURR'] = prvs_app['SK_ID_CURR']
cats_prev.head()

grp = cats_prev.groupby('SK_ID_CURR').mean().reset_index()
grp.columns = ['PREV_'+column if column != 'SK_ID_CURR' else column for column in grp.columns]
apptrain_bureau_prev = apptrain_bureau_prev.merge(grp, on=['SK_ID_CURR'], how='left')
apptrain_bureau_prev.update(apptrain_bureau_prev[grp.columns].fillna(0))

#### 3.3.1.3 Joining pos_cash w/ apptrain_bureau_prev

In [ ]:
print('Shape dari pos_cash:', pos_cash.shape)
pos_cash.head()

Shape dari ps_cash: (3656073, 8)


,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,1803195,182943,-31,48.0,45.0,Active,0.0,0.0
1,1715348,367990,-33,36.0,35.0,Active,0.0,0.0
2,1784872,397406,-32,12.0,9.0,Active,0.0,0.0
3,1903291,269225,-35,48.0,42.0,Active,0.0,0.0
4,2341044,334279,-35,36.0,35.0,Active,0.0,0.0


In [ ]:
# Combining numerical features
grp = pos_cash.drop('SK_ID_PREV', axis =1).groupby(by=['SK_ID_CURR']).mean().reset_index()
prev_columns = ['POS_'+column if column != 'SK_ID_CURR' else column for column in grp.columns ]
grp.columns = prev_columns

apptrain_bureau_prev = apptrain_bureau_prev.merge(grp, on =['SK_ID_CURR'], how = 'left')
apptrain_bureau_prev.update(apptrain_bureau_prev[grp.columns].fillna(0))

In [ ]:
# Combining categorical features
cats_ps_cash = pd.get_dummies(pos_cash.select_dtypes('object'))
cats_ps_cash['SK_ID_CURR'] = pos_cash['SK_ID_CURR']

grp = cats_ps_cash.groupby('SK_ID_CURR').mean().reset_index()

grp.columns = ['POS_'+column if column != 'SK_ID_CURR' else column for column in grp.columns]
apptrain_bureau_prev = apptrain_bureau_prev.merge(grp, on=['SK_ID_CURR'], how='left')
apptrain_bureau_prev.update(apptrain_bureau_prev[grp.columns].fillna(0))

#### 3.3.1.4 Joining ins_pay w/ apptrain_bureau_prev

In [ ]:
# Combining numerical features and there are no categorical features in this dataset
grp = ins_pay.drop('SK_ID_PREV', axis =1).groupby(by=['SK_ID_CURR']).mean().reset_index()
prev_columns = ['INSTA_'+column if column != 'SK_ID_CURR' else column for column in grp.columns ]
grp.columns = prev_columns

apptrain_bureau_prev = apptrain_bureau_prev.merge(grp, on =['SK_ID_CURR'], how = 'left')
apptrain_bureau_prev.update(apptrain_bureau_prev[grp.columns].fillna(0))

#### 3.3.1.5 Joining cc_blnc w/ apptrain_bureau_prev

In [ ]:
# Combining numerical features
grp = cc_blnc.drop('SK_ID_PREV', axis =1).groupby(by=['SK_ID_CURR']).mean().reset_index()
prev_columns = ['CREDIT_'+column if column != 'SK_ID_CURR' else column for column in grp.columns ]
grp.columns = prev_columns

apptrain_bureau_prev = apptrain_bureau_prev.merge(grp, on =['SK_ID_CURR'], how = 'left')
apptrain_bureau_prev.update(apptrain_bureau_prev[grp.columns].fillna(0))

# Combining categorical features
cats_cred = pd.get_dummies(cc_blnc.select_dtypes('object'))
cats_cred['SK_ID_CURR'] = cc_blnc['SK_ID_CURR']
grp = cats_cred.groupby('SK_ID_CURR').mean().reset_index()
grp.columns = ['CREDIT_'+column if column != 'SK_ID_CURR' else column for column in grp.columns]
apptrain_bureau_prev = apptrain_bureau_prev.merge(grp, on=['SK_ID_CURR'], how='left')
apptrain_bureau_prev.update(apptrain_bureau_prev[grp.columns].fillna(0))

### 3.3.2 Data app_test

#### 3.3.2.1 Joining app_test w/ bureau

In [ ]:
# Combining numerical features
grp = bureau.drop(['SK_ID_BUREAU'], axis = 1).groupby(by=['SK_ID_CURR']).mean().reset_index()
grp.columns = ['BUREAU_'+ column if column !='SK_ID_CURR' else column for column in grp.columns]
apptest_bureau = app_test.merge(grp, on='SK_ID_CURR', how='left')
apptest_bureau.update(apptest_bureau[grp.columns].fillna(0))

In [ ]:
#combining categorical features
cats_br = pd.get_dummies(bureau.select_dtypes('object'))
cats_br['SK_ID_CURR'] = bureau['SK_ID_CURR']

grp = cats_br.groupby(by = ['SK_ID_CURR']).mean().reset_index()
grp.columns = ['BUREAU_'+column if column !='SK_ID_CURR' else column for column in grp.columns]
apptest_bureau = apptest_bureau.merge(grp, on='SK_ID_CURR', how='left')
apptest_bureau.update(apptest_bureau[grp.columns].fillna(0))

In [ ]:
# Shape of application and bureau data combined
print('The shape application and bureau data combined:',apptest_bureau.shape)

The shape application and bureau data combined: (47772, 109)


**Feature Engineering of app_bureau Data:**

In [ ]:
# Number of past loans per customer
grp = bureau.groupby(by = ['SK_ID_CURR'])['SK_ID_BUREAU'].count().reset_index().rename(columns = {'SK_ID_BUREAU': 'BUREAU_LOAN_COUNT'})

apptest_bureau = apptest_bureau.merge(grp, on='SK_ID_CURR', how='left')
apptest_bureau['BUREAU_LOAN_COUNT'] = apptest_bureau['BUREAU_LOAN_COUNT'].fillna(0)

In [ ]:
# Number of types of past loans per customer 
grp = bureau[['SK_ID_CURR', 'CREDIT_TYPE']].groupby(by = ['SK_ID_CURR'])['CREDIT_TYPE'].nunique().reset_index().rename(columns={'CREDIT_TYPE': 'BUREAU_LOAN_TYPES'})

apptest_bureau = apptest_bureau.merge(grp, on='SK_ID_CURR', how='left')
apptest_bureau['BUREAU_LOAN_TYPES'] = apptest_bureau['BUREAU_LOAN_TYPES'].fillna(0)

In [ ]:
# Debt over credit ratio 
bureau['AMT_CREDIT_SUM'] = bureau['AMT_CREDIT_SUM'].fillna(0)
bureau['AMT_CREDIT_SUM_DEBT'] = bureau['AMT_CREDIT_SUM_DEBT'].fillna(0)

grp1 = bureau[['SK_ID_CURR','AMT_CREDIT_SUM']].groupby(by=['SK_ID_CURR'])['AMT_CREDIT_SUM'].sum().reset_index().rename(columns={'AMT_CREDIT_SUM': 'TOTAL_CREDIT_SUM'})
grp2 = bureau[['SK_ID_CURR','AMT_CREDIT_SUM_DEBT']].groupby(by=['SK_ID_CURR'])['AMT_CREDIT_SUM_DEBT'].sum().reset_index().rename(columns={'AMT_CREDIT_SUM_DEBT':'TOTAL_CREDIT_SUM_DEBT'})

grp1['DEBT_CREDIT_RATIO'] = grp2['TOTAL_CREDIT_SUM_DEBT']/grp1['TOTAL_CREDIT_SUM']
del grp1['TOTAL_CREDIT_SUM']

apptest_bureau = apptest_bureau.merge(grp1, on='SK_ID_CURR', how='left')
apptest_bureau['DEBT_CREDIT_RATIO'] = apptest_bureau['DEBT_CREDIT_RATIO'].fillna(0)
apptest_bureau['DEBT_CREDIT_RATIO'] = apptest_bureau['DEBT_CREDIT_RATIO'].replace([np.inf, -np.inf], 0)
apptest_bureau['DEBT_CREDIT_RATIO'] = pd.to_numeric(apptest_bureau['DEBT_CREDIT_RATIO'], downcast='float')

In [ ]:
# Overdue over debt ratio
bureau['AMT_CREDIT_SUM_OVERDUE'] = bureau['AMT_CREDIT_SUM_OVERDUE'].fillna(0)
bureau['AMT_CREDIT_SUM_DEBT'] = bureau['AMT_CREDIT_SUM_DEBT'].fillna(0)

grp1 = bureau[['SK_ID_CURR','AMT_CREDIT_SUM_OVERDUE']].groupby(by=['SK_ID_CURR'])['AMT_CREDIT_SUM_OVERDUE'].sum().reset_index().rename(columns={'AMT_CREDIT_SUM_OVERDUE': 'TOTAL_CUSTOMER_OVERDUE'})
grp2 = bureau[['SK_ID_CURR','AMT_CREDIT_SUM_DEBT']].groupby(by=['SK_ID_CURR'])['AMT_CREDIT_SUM_DEBT'].sum().reset_index().rename(columns={'AMT_CREDIT_SUM_DEBT':'TOTAL_CUSTOMER_DEBT'})

grp1['OVERDUE_DEBT_RATIO'] = grp1['TOTAL_CUSTOMER_OVERDUE']/grp2['TOTAL_CUSTOMER_DEBT']
del grp1['TOTAL_CUSTOMER_OVERDUE']

apptest_bureau = apptest_bureau.merge(grp1, on='SK_ID_CURR', how='left')
apptest_bureau['OVERDUE_DEBT_RATIO'] =apptest_bureau['OVERDUE_DEBT_RATIO'].fillna(0)
apptest_bureau['OVERDUE_DEBT_RATIO'] = apptest_bureau['OVERDUE_DEBT_RATIO'].replace([np.inf, -np.inf], 0)
apptest_bureau['OVERDUE_DEBT_RATIO'] = pd.to_numeric(apptest_bureau['OVERDUE_DEBT_RATIO'], downcast='float')

#### 3.3.2.2 Joining prvs_app w/ apptest_bureau

In [ ]:
# Number of previous applications per customer
grp = prvs_app[['SK_ID_CURR','SK_ID_PREV']].groupby(by=['SK_ID_CURR'])['SK_ID_PREV'].count().reset_index().rename(columns={'SK_ID_PREV':'PREV_APP_COUNT'})
apptest_bureau_prev = apptest_bureau.merge(grp, on =['SK_ID_CURR'], how = 'left')
apptest_bureau_prev['PREV_APP_COUNT'] = apptest_bureau_prev['PREV_APP_COUNT'].fillna(0)

In [ ]:
# Combining numerical features
grp = prvs_app.drop('SK_ID_PREV', axis =1).groupby(by=['SK_ID_CURR']).mean().reset_index()
prev_columns = ['PREV_'+column if column != 'SK_ID_CURR' else column for column in grp.columns ]
grp.columns = prev_columns
apptest_bureau_prev = apptest_bureau_prev.merge(grp, on =['SK_ID_CURR'], how = 'left')
apptest_bureau_prev.update(apptest_bureau_prev[grp.columns].fillna(0))

In [ ]:
# Combining categorical features
cats_prev = pd.get_dummies(prvs_app.select_dtypes('object'))
cats_prev['SK_ID_CURR'] = prvs_app['SK_ID_CURR']
cats_prev.head()

grp = cats_prev.groupby('SK_ID_CURR').mean().reset_index()
grp.columns = ['PREV_'+column if column != 'SK_ID_CURR' else column for column in grp.columns]
apptest_bureau_prev = apptest_bureau_prev.merge(grp, on=['SK_ID_CURR'], how='left')
apptest_bureau_prev.update(apptest_bureau_prev[grp.columns].fillna(0))

#### 3.3.2.3 Joining pos_cash w/ apptest_bureau_prev

In [ ]:
print('Shape dari ps_cash:', pos_cash.shape)
pos_cash.head()

Shape dari ps_cash: (3656073, 8)


,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,1803195,182943,-31,48.0,45.0,Active,0.0,0.0
1,1715348,367990,-33,36.0,35.0,Active,0.0,0.0
2,1784872,397406,-32,12.0,9.0,Active,0.0,0.0
3,1903291,269225,-35,48.0,42.0,Active,0.0,0.0
4,2341044,334279,-35,36.0,35.0,Active,0.0,0.0


In [ ]:
# Combining numerical features
grp = pos_cash.drop('SK_ID_PREV', axis =1).groupby(by=['SK_ID_CURR']).mean().reset_index()
prev_columns = ['POS_'+column if column != 'SK_ID_CURR' else column for column in grp.columns ]
grp.columns = prev_columns

apptest_bureau_prev = apptest_bureau_prev.merge(grp, on =['SK_ID_CURR'], how = 'left')
apptest_bureau_prev.update(apptest_bureau_prev[grp.columns].fillna(0))

In [ ]:
# Combining categorical features
cats_ps_cash = pd.get_dummies(pos_cash.select_dtypes('object'))
cats_ps_cash['SK_ID_CURR'] = pos_cash['SK_ID_CURR']

grp = cats_ps_cash.groupby('SK_ID_CURR').mean().reset_index()

grp.columns = ['POS_'+column if column != 'SK_ID_CURR' else column for column in grp.columns]
apptest_bureau_prev = apptest_bureau_prev.merge(grp, on=['SK_ID_CURR'], how='left')
apptest_bureau_prev.update(apptest_bureau_prev[grp.columns].fillna(0))

#### 3.3.2.4 Joining ins_pay w/ apptest_bureau_prev

In [ ]:
# Combining numerical features and there are no categorical features in this dataset
grp = ins_pay.drop('SK_ID_PREV', axis =1).groupby(by=['SK_ID_CURR']).mean().reset_index()
prev_columns = ['INSTA_'+column if column != 'SK_ID_CURR' else column for column in grp.columns ]
grp.columns = prev_columns

apptest_bureau_prev = apptest_bureau_prev.merge(grp, on =['SK_ID_CURR'], how = 'left')
apptest_bureau_prev.update(apptest_bureau_prev[grp.columns].fillna(0))

#### 3.3.2.5 Joining cc_blnc w/ app_bureau_prev

In [ ]:
# Combining numerical features
grp = cc_blnc.drop('SK_ID_PREV', axis =1).groupby(by=['SK_ID_CURR']).mean().reset_index()
prev_columns = ['CREDIT_'+column if column != 'SK_ID_CURR' else column for column in grp.columns ]
grp.columns = prev_columns

apptest_bureau_prev = apptest_bureau_prev.merge(grp, on =['SK_ID_CURR'], how = 'left')
apptest_bureau_prev.update(apptest_bureau_prev[grp.columns].fillna(0))

# Combining categorical features
cats_cred = pd.get_dummies(cc_blnc.select_dtypes('object'))
cats_cred['SK_ID_CURR'] = cc_blnc['SK_ID_CURR']
grp = cats_cred.groupby('SK_ID_CURR').mean().reset_index()
grp.columns = ['CREDIT_'+column if column != 'SK_ID_CURR' else column for column in grp.columns]
apptest_bureau_prev = apptest_bureau_prev.merge(grp, on=['SK_ID_CURR'], how='left')
apptest_bureau_prev.update(apptest_bureau_prev[grp.columns].fillna(0))

## 3.4 Checking the Data

In [ ]:
apptrain_bureau_prev.shape

(304527, 323)

In [ ]:
apptest_bureau_prev.shape

(47772, 322)